# Backdoor & Vulnerability Creation Kill Chain

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

**Threat Actor Profile (TAP):** 003 <br/>
**Kill Chain**: Backdoor & Vulnerability Creation

This notebook demonstrates a new UC7 Kill Chain which aims to represent a different style of attack in comparison to the mobile malware kill chain (TAP001).
Kill chains aim to represent a potential real world cyber attack. In this scenario an malicious `SOME_TECH` admin (TAP003) leverages his legitimate credentials and permissions to create purposeful backdoors, establish footholds and use other legitimate features in malicious ways.
<br/>

In this version - this scenario is limited in scope. TAP003 opts to alter user accounts and implement malicious ACL rule by using the ``terminal`` service to SSH into target routers. These ACLs block green traffic which trigger a negative reward.
<br/>

This kill chain intends to introduce a new UC7 attack which is both realistic but also dissimilar to other kill chains.

In [ ]:
!primaite setup

In [ ]:
# Importing the necessary primAITE libraries
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.services.dns.dns_client import DNSClient
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.system.applications.web_browser import WebBrowser
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.config.load import load, _EXAMPLE_CFG
from pprint import pprint
from deepdiff.diff import DeepDiff
from prettytable import PrettyTable
import yaml

In [ ]:
# Utility functions.

def print_agent_actions_except_do_nothing(agent_name):
    """Get the agent's action history, filter out `do-nothing` actions, print relevant data in a table."""
    table = PrettyTable()
    table.field_names = ["Step", "Action", "Node", "Application", "Target IP", "Response"]
    print(f"Episode: {env.episode_counter}, Actions for '{agent_name}':")
    for item in env.game.agents[agent_name].history:
        if item.action == "do-nothing":
            continue

        node, application, target_ip = "N/A", "N/A", "N/A",

        if item.action.startswith("node-nmap"):
            node = item.parameters['source_node']
            application = "nmap"
            target_ip = str(item.parameters['target_ip_address'])
            target_ip = (target_ip[:25]+'...') if len(target_ip)>25 else target_ip # truncate long string

        elif item.action == "router-acl-add-rule":
            node = item.parameters.get("router_name")
        elif item.action == "node-send-remote-command":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = item.parameters.get("command")
        elif item.action == "node-session-remote-login":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = "user-manager"
        elif item.action.startswith("c2-server"):
            application = "c2-server"
            node = item.parameters.get('node_name')
        elif item.action == "configure-c2-beacon":
            application = "c2-beacon"
            node = item.parameters.get('node_name')

        else:
            if (node_id := item.parameters.get('node_id')) is not None:
                node = env.game.agents[agent_name].action_manager.node_names[node_id]
            if (application_id := item.parameters.get('application_id')) is not None:
                application = env.game.agents[agent_name].action_manager.application_names[node_id][application_id]
            if (application_name := item.parameters.get('application_name')) is not None:
                application = application_name

        table.add_row([item.timestep, item.action, node, application, target_ip, item.response.status])

    print(table)
    print("(Any do-nothing actions are omitted)")


In [ ]:
def display_obs_diffs(old, new, step_counter):
    """
    Use DeepDiff to extract and display differences in old and new instances of
    the observation space.

    :param old: observation space instance.
    :param new: observation space instance.
    :param step_counter: current step counter.
    """
    print("\nObservation space differences")
    print("-----------------------------")
    diff = DeepDiff(old, new)
    print(f"Step {step_counter}")
    for d,v in diff.get('values_changed', {}).items():
        print(f"{d}: {v['old_value']} -> {v['new_value']}")




### **Notebook Sections:**
1. Notebook Intro
2. Notebook Setup
3. Prior To Attack
4. Kill Chain Stage Demonstration
5. Attack Configurations

### **Notebook Intro** | **Backdoor & Vulnerability Creation Kill Chain Intro** 

TAP003's kill chain is comprised of a variety of blue actions which are leveraged in unusual ways. This includes introducing malicious ACL's which block green traffic and installing and execute green applications in order to simulate unusual green pattern of life. The rest of this notebook will go through each step in more detail whilst demonstrating the impacts that each step has on both observation and simulation behaviour.

_Reconnaissance - DONOTHING CAOS Action_

|Index | Action Stage| OBS Impact | Narrative |
|-----|-------------|-------------|-----------|
|1|Reconnaissance|*No Direct Impact*|TAP003 is passively investigating sensitive systems, data and access control mechanisms.|

_Planning - DONOTHING CAOS Action_

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|2|Planning| **No current impact**|TAP003 is devising a plan to exploit their elevated privileges.|

 _Access - DONOTHING CAOS Action__

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|3|Access|**No current impact** |TAP003 uses their legitimate credentials to access the access control settings.|

 _Manipulation - HOST:SESSIONS_SEND_REMOTE_COMMAND -> HOST:ACCOUNTS:CHANGE:PASSWORD CAOS ACTION_
 
|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|4|Manipulation| **Target Host(s)** HOST::SESSIONS:REMOTE |TAP003 exploits their insider knowledge/privilege to implement changes for sabotage.|

 _Exploit - FIREWALL:ACL:add_rule CAOS ACTION_

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|5|Exploit| **Target Host(s)** FIREWALL:ACL:INTERNAL/EXTERNAL:*|TAP003 exploits their insider knowledge/privilege to implement changes for sabotage.|

_Only the initial five steps are represented in the this version of this kill-chain._ <br/>


## **Notebook Setup** | **Network Configuration:**

This notebook uses the same network setup as UC7. 

Please refer to the main [UC7-E2E-Demo notebook for further reference](./UC7-E2E-Demo.ipynb).

<p align="center">
    <a href="./_package_data/uc7/uc7_tap003/uc7_tap003_main.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap003/uc7_tap003_main.png" alt="Image" style="width:1000px;">
    </a>  
    
</p>

_(Click to enlarge)_

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"][33]["agent_settings"]["flatten_obs"] = False
    cfg['io_settings']['save_agent_logs'] = True
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
env = PrimaiteGymEnv(env_config=cfg)
env.game.simulation.network.show()

### **Notebook Setup** | Instantiating Relevant Simulation Objects
The cell below sets up the relevant simulation and agent objects for the kill chain demonstration.


The following cell resets the environment, instantiates TAP003 and it's selected starting/target hosts.

In [ ]:
env.reset() # resetting the environment
# The TAP003 Agent
tap003 = env.game.agents['attacker']
tap003.logger.logger.setLevel("INFO")

## **Prior To Attack** | Initial States:

The first section of this notebook displays the initial simulation state of TAP003's starting and target nodes.

#### **Initial State** | Starting Host:
TAP003's initial starting point.

TAP003's starting host does not cause any MNE (Malicious Network Events) in it's current implementation.

In [ ]:
# TAP003's Starting Client:
starting_host = env.game.simulation.network.get_node_by_hostname(tap003.starting_node)
print(f"Starting host:")
starting_host.show()
starting_host.software_manager.show()

#### **Initial State** | Target Hosts

TAP003's selected target hosts at current the only valid targets for TAP003 are Router/Firewall type nodes

Code snippet below shows the default OBS of the target routers

In [ ]:
# TAP003's target routers

st_intra_prv_rt_cr: Router = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_dr_1: Router = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1")
rem_pub_rt_dr: Router = env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR")

st_intra_prv_rt_cr.acl.show()
st_intra_prv_rt_dr_1.acl.show()
rem_pub_rt_dr.acl.show()

## **Kill Chain** | Kill Chain Stage Demonstration 

For the initial kill chain demonstration, TAP003 is configured with the following yaml snippet:

```yaml
  - ref: attacker
    team: RED
    type: tap-003
    observation_space: {}
    action_space: {}
    agent_settings:
      start_step: 1
      frequency: 3
      variance: 0
      repeat_kill_chain: false
      repeat_kill_chain_stages: true
      default_starting_node: "ST_PROJ-A-PRV-PC-1"
      starting_nodes:
      # starting_nodes: ["ST_PROJ-A-PRV-PC-1", "ST_PROJ-B-PRV-PC-2", "ST_PROJ-C-PRV-PC-3"]
      kill_chain:
        PLANNING:
          probability: 1
          starting_network_knowledge:
            credentials:
              ST_PROJ-A-PRV-PC-1:
                username: admin
                password: admin
              ST_PROJ-B-PRV-PC-2:
                username: admin
                password: admin
              ST_PROJ-C-PRV-PC-3:
                username: admin
                password: admin
              ST_INTRA-PRV-RT-DR-1:
                ip_address: 192.168.230.1
                username: admin
                password: admin
              ST_INTRA-PRV-RT-CR:
                ip_address: 192.168.160.1
                username: admin
                password: admin
              REM-PUB-RT-DR:
                ip_address: 192.168.10.2
                username: admin
                password: admin
        ACCESS:
          probability: 1
        MANIPULATION:
          probability: 1
          account_changes:
            - host: ST_INTRA-PRV-RT-DR-1
              ip_address: 192.168.230.1 # ST_INTRA-PRV-RT-DR-1
              action: change_password
              username: admin
              new_password: "red_pass"
            - host: ST_INTRA-PRV-RT-CR
              ip_address: 192.168.160.1 # ST_INTRA-PRV-RT-CR
              action: change_password
              username: "admin"
              new_password: "red_pass"
            - host: REM-PUB-RT-DR
              ip_address: 192.168.10.2 # REM-PUB-RT-DR
              action: change_password
              username: "admin"
              new_password: "red_pass"
        EXPLOIT:
          probability: 1
          malicious_acls:
            - target_router: ST_INTRA-PRV-RT-DR-1
              position: 1
              permission: DENY
              src_ip: ALL
              src_wildcard: 0.0.255.255
              dst_ip: ALL
              dst_wildcard: 0.0.255.255
              src_port: POSTGRES_SERVER
              dst_port: POSTGRES_SERVER
              protocol_name_name: TCP
            - target_router: ST_INTRA-PRV-RT-CR
              position: 1
              permission: DENY
              src_ip: ALL
              src_wildcard: 0.0.255.255
              dst_ip: ALL
              dst_wildcard: 0.0.255.255
              src_port: HTTP
              dst_port: HTTP
              protocol_name_name: TCP
            - target_router: REM-PUB-RT-DR
              position: 1
              permission: DENY
              src_ip: ALL
              src_wildcard: 0.0.255.255
              dst_ip: ALL
              dst_wildcard: 0.0.255.255
              src_port: DNS
              dst_port: DNS
              protocol_name_name: TCP

```
_Further information & guidance around configuring TAP003 can be found within the last section of this notebook._

### **Kill Chain** | NOT STARTED
This is the default state of the TAP003 Agent. This stage indicates that the TAP003 agent has not begun it's kill chain.
If TAP003 is in this stage then on the next execution step the kill chain will begin.

In [ ]:
tap003 = env.game.agents['attacker']


print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")
while(tap003.current_kill_chain_stage == InsiderKillChain.NOT_STARTED):
    default_obs, _, _, _, _ = env.step(0)
starting_host.software_manager.show()

env.step(0);

In [ ]:
tap003.logger.show()

### **Kill Chain** | RECONNAISSANCE

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|1|RECONNAISSANCE|*No Direct Impact*|TAP003 is identifying Sensitive systems, data and access control mechanisms in legitimate ways.|

Currently, this stage in the kill chain is implemented via the 'DONOTHING' CAOS action.

In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")

while(tap003.current_kill_chain_stage == InsiderKillChain.RECONNAISSANCE):
    reconnaissance_obs_impact, _, _, _, _ = env.step(0)

current_host = env.game.simulation.network.get_node_by_hostname(tap003.current_host)
current_host.software_manager.show()
display_obs_diffs(default_obs, reconnaissance_obs_impact, env.game.step_counter)

In [ ]:
tap003.logger.show()

### **Kill Chain** | PLANNING

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|2|PLANNING| *No Direct Impact*| HOST:APPLICATION|TAP003 is devising a plan to exploit their elevated privileges.|

Currently, the TAP003 agent does not perform any actions at this stage of the kill chain.

In [ ]:
pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")
pprint("---Starting Client State----")
while(tap003.current_kill_chain_stage == InsiderKillChain.PLANNING):
    planning_obs_impact, _, _, _, info = env.step(0);
current_host.software_manager.show()
display_obs_diffs(reconnaissance_obs_impact, planning_obs_impact, env.game.step_counter)

In [ ]:
tap003.logger.show()

### **Kill Chain** | ACCESS

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|3|Access|_DONOTHING CAOS Action_|TAP003 uses their legitimate credentials to access the access control settings.|

Currently, at this point of the kill chain stage the TAP003 does not perform any simulations actions. Future versions of TAP003 aim to leverage more of the simulation to create and remove accounts at this stage.

In [ ]:
pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")
pprint("---Starting Client State----")
while(tap003.current_kill_chain_stage == InsiderKillChain.ACCESS):
    access_obs_impact, _, _, _, info = env.step(0)



In [ ]:

# Compare observation space before and after ACCESS stage.
display_obs_diffs(planning_obs_impact, access_obs_impact, env.game.step_counter)

starting_host.software_manager.show()
print_agent_actions_except_do_nothing(tap003.config.ref)

In [ ]:
tap003.logger.show()

### **Kill Chain** | MANIPULATION

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|4|Manipulation| **Target Host(s)** HOST::SESSIONS:REMOTE |TAP003 exploits their insider knowledge/privilege to implement changes for sabotage.|

In the ``MANIPULATION`` kill chain stage the TAP003 agent creates new and alters pre-existing user accounts based on user given settings. 

_For more information please refer to the later TAP003 customisation section of the notebook._

In [ ]:
access_obs_impact, _, _, _, _ = env.step(0)
pprint(f"Agent action: {info['agent_actions']['attacker']}")

pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")
pprint("---Target Client State----")
while(tap003.current_kill_chain_stage == InsiderKillChain.MANIPULATION):
    manipulation_obs_impact, _, _, _, info = env.step(0)


# Example host
rem_pub_rt_dr.user_manager.show()
# rem_pub_rt_dr.user_session_manager.show()

# Compare observation space before and after MANIPULATION stage.
display_obs_diffs(access_obs_impact, manipulation_obs_impact, env.game.step_counter)
print_agent_actions_except_do_nothing(tap003.config.ref)

In [ ]:
tap003.logger.show()

### **Kill Chain** | EXPLOITATION

|Index| Action Stage| OBS Impact | Narrative |
|-----|-------------|------------|-----------|
|5|EXPLOITATION| **Target Host(s)** ROUTER_ACL_ADDRULE |TAP003 exploiting their insider knowledge/privilege to implement changes for sabotage|

In the final attack stage the TAP003 agent insert a malicious ACL rule(s) which blocks specific traffic, therefore triggering negative rewards for the green agents.

The table below shows the impacts:

|Target Router         | Impact |
|----------------------|--------|
|`ST_INTRA-PRV-RT-DR-1`| Blocks all `POSTGRES_SERVER` that arrives at the `ST_INTRA-PRV-RT-DR-1` router. This rule will prevent all ST_PROJ_* hosts from accessing the database (`ST_DATA-PRV-SRV-DB`).|
|`ST_INTRA-PRV-RT-CR`| Blocks all `HTTP` traffic that arrives at the`ST_INTRA-PRV-RT-CR` router. This rule will prevent all SOME_TECH hosts from accessing the webserver (`ST_DMZ-PUB-SRV-WEB`)|
|`REM-PUB-RT-DR`| Blocks all `DNS` traffic that arrives at the `REM-PUB-RT-DR` router. This rule prevents any remote site works from accessing the DNS Server (`ISP-PUB-SRV-DNS`).|


<p align="center">
    <a href="./_package_data/uc7/uc7_tap003/uc7_tap003_main.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap003/uc7_tap003_main.png" alt="Image" style="width:1000px;">
    </a>  
    
</p>

_(Click to enlarge)_

In [ ]:
manipulation_obs_impact, _, _, _, _ = env.step(0)
pprint(f"Agent action: {info['agent_actions']['attacker']}")

pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")
pprint("---Target Client State----")
while(tap003.current_kill_chain_stage == InsiderKillChain.EXPLOIT):
    exploit_obs_impact, _, _, _, info = env.step(0)
    
st_intra_prv_rt_cr.acl.show()
st_intra_prv_rt_dr_1.acl.show()
rem_pub_rt_dr.acl.show()

# Stepping once more to allow the malicious acl to appear in the simulation.
exploit_obs_impact, _, _, _, _ = env.step(0)

# Compare observation space before and after MANIPULATION stage.
display_obs_diffs(manipulation_obs_impact, exploit_obs_impact, env.game.step_counter)
print_agent_actions_except_do_nothing(tap003.config.ref)

In [ ]:
tap003.logger.show()

#### Demonstration of the ACL Rule blocking traffic

As an example of the malicious ACL Rule affecting traffic across the network, attempting to query the database server (ST_DATA-PRV-SRV-DB) from any of the `ST_PROJECT_*` networks should fail because it is must route through the `ST_INTRA-PRV-RT-DR-1` router which TAP003 has configured to block all `POSTGRES_SERVER` traffic.

In [ ]:
affected_node = env.game.simulation.network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
database_server = env.game.simulation.network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")

st_intra_prv_rt_dr_1 = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1")

database_client: DatabaseClient = affected_node.software_manager.software.get("database-client")
st_intra_prv_rt_dr_1.acl.show()

# should be False
print(database_client.query(sql="SELECT"))

However, ACL rules will not stop ICMP traffic, allowing us to still ping the target nodes as shown below.

In [ ]:
# should be true
print(affected_node.ping(database_server.network_interface[1].ip_address))

Additionally, by default, TAP003 is configured to add two more ACLs on the UC7 network as shown in the table previously.

In [ ]:
# This ACL blocks all web_server traffic from within the main `SOME_TECH` office. Thus leading to every SOME_TECH host unable to access the web-server
st_intra_prv_rt_cr = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_cr.acl.show()

st_head_office_private_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("ST_HO-PRV-PC-1")
st_head_office_private_pc_1_web_browser: WebBrowser = st_head_office_private_pc_1.software_manager.software["web-browser"]
st_head_office_private_pc_1_web_browser.get_webpage(url="http://some_tech.com")
st_head_office_private_pc_1_web_browser.sys_log.show()

In [ ]:
# This ACL blocks all DNS traffic from exiting the SOME_TECH remote site.
rem_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname(hostname="REM-PUB-PC-1")
rem_pub_rt_dr = env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR")
dns_client: DNSClient = rem_pub_pc_1.software_manager.software["dns-client"]
dns_client.dns_cache.clear()
dns_client.check_domain_exists(target_domain="some_tech.com")
rem_pub_rt_dr.acl.show()

## **Kill Chain Configurations** | Overview:

The following sections cover the TAP003's settings & configurations. 
Each section contains code cells which directly demonstrate each option and it's effect on TAP003.

### **Attack Configurations** | Threat Actor Profile (TAP) wide Agent Settings

<summary> Threat Actor Profile Agent Settings </summary>

<details>

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|start_step|The initial kill chain starting step.|Int|_Required_|
|frequency|The frequency of each kill chains starting steps.|Int|_Required_|
|variance| The timestep variance between frequency|Int|_Required_|
|repeat_kill_chain|Indicates whether the attack is repeated throughout the episode.|Bool|_Required_|
|repeat_kill_chain_stages|Indicates if the kill_chain stage should reset upon failure or retry.|Bool|_Required_|
|default_target_node|The Target Host|Str|_Required_|
|starting_nodes|A list of Potential Targets|List|_Optional_|
|kill_chain|_See the next notebook section_|Dict|_Required_||

</details>

_Example kill chain setting configuration_
```yaml
      start_step: 1
      frequency: 3
      variance: 0
      repeat_kill_chain: false
      repeat_kill_chain_stages: true
      default_starting_node: "ST_PROJ-A-PRV-PC-1"
      starting_nodes:
      # starting_nodes: ["ST_PROJ-A-PRV-PC-1", "ST_PROJ-B-PRV-PC-2", "ST_PROJ-C-PRV-PC-3"]
```

### **Attack Configurations** | Kill Chain Settings Overview
Currently, each kill chain stage has a probability of success option.
Additionally the manipulation kill stage offers an additional configuration of the malicious ACL.

<summary> TAP003's Kill Chain Settings </summary>

<details> 

Kill Chain Stage 1 - _PLANNING_

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Probability Of Success - The chance of successfully carrying out this stage in the kill_chain.|float|_Required_|
|starting_network_knowledge| User Account details | dict |_required_|

Kill Chain Stage 2 - _ACCESS_

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Probability Of Success - The chance of successfully carrying out this stage in the kill_chain.|float|_Required_|

Kill Chain Stage 3 - _MANIPULATION_

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Probability Of Success - The chance of successfully carrying out this stage in the kill_chain.|float|_Required_|
|account_changes| User Accounts hijacked - Must be the same targets as `malicious_acl` used in next kill chain stage | dict | _Required_|

Kill Chain Stage 4 - _EXPLOIT_

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|malicious_acl|The configurable ACL that the TAP003 agent adds to the target node.|dict|_Required_|

</details>

_An example of a kill chain setting configuration_ 
```yaml
        kill_chain:
          PLANNING:
            probability: 1
            starting_network_knowledge:
              credentials:
                ST_PROJ-A-PRV-PC-1:
                  username: admin
                  password: admin
                ST_PROJ-B-PRV-PC-2:
                  username: admin
                  password: admin
                ST_PROJ-C-PRV-PC-3:
                  username: admin
                  password: admin
                ST_INTRA-PRV-RT-DR-1:
                  ip_address: 192.168.230.1
                  username: admin
                  password: admin
                ST_INTRA-PRV-RT-CR:
                  ip_address: 192.168.160.1
                  username: admin
                  password: admin
                REM-PUB-RT-DR:
                  ip_address: 192.168.10.2
                  username: admin
                  password: admin
          ACCESS:
            probability: 1
          MANIPULATION:
            probability: 1
            account_changes:
              - host: ST_INTRA-PRV-RT-DR-1
                ip_address: 192.168.230.1 # ST_INTRA-PRV-RT-DR-1
                action: change_password
                username: admin
                new_password: "red_pass"
              - host: ST_INTRA-PRV-RT-CR
                ip_address: 192.168.160.1 # ST_INTRA-PRV-RT-CR
                action: change_password
                username: "admin"
                new_password: "red_pass"
              - host: REM-PUB-RT-DR
                ip_address: 192.168.10.2 # REM-PUB-RT-DR
                action: change_password
                username: "admin"
                new_password: "red_pass"
          EXPLOIT:
            probability: 1
            malicious_acls:
              - target_router: ST_INTRA-PRV-RT-DR-1
                ip_address: 192.168.230.1 
                position: 1
                permission: DENY
                src_ip: ALL
                src_wildcard_mask: 0.0.255.255
                dst_ip: ALL
                dst_wildcard_mask: 0.0.255.255
                src_port: POSTGRES_SERVER
                dst_port: POSTGRES_SERVER
                protocol_name: TCP

```

### **Attack Configurations** | Configuration Examples 
The following notebook section demonstrates a few potential examples of different TAP003 configurations and their expected behaviour within an episode.

In [ ]:
tap003 = env.game.agents['attacker']
for key,value in tap003.config.agent_settings:
    if key == 'kill_chain':
        pass
    else:
        print(f"{key} : {value}")

#### **Attack Configurations** | Start Step, Frequency & Variance

In this section, the code cells below demonstrate the following options:

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|start_step|The initial kill chain starting step.|Int|_Required_|
|frequency|The frequency of each kill chains starting steps.|Str|_Required_|
|variance|The timestep variance between frequency.|Str|_Required_|


The code cell below alters the start_step, frequency and variance to 20, 10 and 5 respectively.
Additionally, the probability of the kill chain stages are set to guarantee success. 

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"][33]["agent_settings"]["flatten_obs"] = False
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['agents'][32]['agent_settings']['start_step'] = 20
    cfg['agents'][32]['agent_settings']['frequency'] = 10
    cfg['agents'][32]['agent_settings']['variance'] = 5
    cfg['agents'][32]['agent_settings']['kill_chain']['MANIPULATION']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['ACCESS']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['PLANNING']['probability'] = 1
env = PrimaiteGymEnv(env_config = cfg)

The cell below demonstrates how the frequency & variance options affect TAP003's next execution step.

In [ ]:
tap003 = env.game.agents['attacker']
print("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")

#### **Attack Configurations** | repeat_kill_chain & repeat_kill_chain_stages

This section demonstrate how the different repeat options may affect a 100 step length episode.

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|repeat_kill_chain|Indicates whether the attack is repeated throughout the episode.|Str|_Required_|
|repeat_kill_chain_stages|Indicates if the kill_chain stage should reset upon failure or retry.|Str|_Required_|


In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
    cfg['agents'][32]['agent_settings']['repeat_kill_chain_stages'] = False
env = PrimaiteGymEnv(env_config = cfg)

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = True
    cfg['agents'][32]['agent_settings']['repeat_kill_chain_stages'] = True
env = PrimaiteGymEnv(env_config = cfg)

In [ ]:
while(env.game.step_counter != 100):
    env.step(0)

#### **Attack Configurations** | Starting Node

The code cells below demonstrate how to configure TAP003's different options for selecting starting hosts.

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|default_starting_node|The default starting host.|Str|_Required_|
|starting_nodes|A list of potential targets|List|_Optional_|


In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg["agents"][32]["agent_settings"]["default_starting_node"] = "ST_PROJ-A-PRV-PC-1"
env = PrimaiteGymEnv(env_config = cfg)
tap003 = env.game.agents["attacker"]
print(f"TA003's Selected Starting Host: {tap003.starting_node}")

The code cell below demonstrates how starting node lists overrule the default start node options:

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg["agents"][32]["agent_settings"]["default_starting_node"] = "ST_PROJ-A-PRV-PC-1"
    cfg["agents"][32]["agent_settings"]["starting_nodes"] = ["ST_PROJ-A-PRV-PC-1","ST_PROJ-B-PRV-PC-2"]

In [ ]:
env = PrimaiteGymEnv(env_config = cfg)
tap003 = env.game.agents["attacker"]
print(f"TA003's Selected Starting Host: {tap003.starting_node}")

#### **Attack Configurations** | kill_chain_settings | probability

The code cells below configure each kill chain's probability of success. 
The cells also demonstrate how TAP003 handles failure within an episode.

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability | The chance of successfully carrying out this stage in the kill chain.|float|_Required_|


The code cell below directly sets each probability of success to 0.25. <Br/>
Additionally the repeat kill chain and kill chain stage configuration options are set to false.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
    cfg['agents'][32]['agent_settings']['repeat_kill_chain_stages'] = False
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['probability'] = 0.25
    cfg['agents'][32]['agent_settings']['kill_chain']['ACCESS']['probability'] = 0.25
    cfg['agents'][32]['agent_settings']['kill_chain']['PLANNING']['probability'] = 0.25
env = PrimaiteGymEnv(env_config = cfg)

In [ ]:
while(env.game.step_counter != 100):
    env.step(0)
tap003 = env.game.agents['attacker']
pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")

The code cell below performs the same episode but with the repeat_kill_chain and repeat_kill_chain_stages to true.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = True
    cfg['agents'][32]['agent_settings']['repeat_kill_chain_stages'] = True
    cfg['agents'][32]['agent_settings']['kill_chain']['MANIPULATION']['probability'] = 0.25
    cfg['agents'][32]['agent_settings']['kill_chain']['ACCESS']['probability'] = 0.25
    cfg['agents'][32]['agent_settings']['kill_chain']['PLANNING']['probability'] = 0.25
env = PrimaiteGymEnv(env_config = cfg)

In [ ]:
while(env.game.step_counter != 100):
    env.step(0)
tap003 = env.game.agents['attacker']
pprint("---TAP003's Current State---")
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap003.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap003.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap003.next_kill_chain_stage.name}")

#### **Attack Configurations** | kill_chain_settings | ACCESS - Starting Network Knowledge

|Option Field |Meaning|Expected Type|Required/Optional|
|-------------|-------|-------------|-----------------|
|probability  |Action Probability - The chance of successfully carrying out this stage in the kill_chain.|str|_Required_|
|starting_network_knowledge|TAP003's starting knowledge. Used to login into the target accounts in the ``MANIPULATION`` stage.|dict|_Required_|

``starting_network_knowledge``

***

The ``starting_network_knowledge`` options uses the following schema:

|Option Field |Meaning|Expected Type|Required/Optional|
|-------------|-------|-------------|-----------------|
|credentials|The credential knowledge does TAP003 starts the episode with.|dict|_Required_|

```yaml
    kill_chain:
        PLANNING:
        probability: 1
        starting_network_knowledge:
            credentials:
```

``credentials``

***

The ``credentials`` options uses the following schema:

|Option Field |Meaning|Expected Type|Required/Optional|
|-------------|-------|-------------|-----------------|
|ip_address   |The IP Address of the target host.|str|_Required_|
|username     |A username of a valid user account. Defaults to ``Admin``.|str|_Optional_|
|password     |The corresponding valid password of the given username. Defaults to ``Admin``.|str|_Optional_|

These credentials will are used in the ``MANIPULATION`` kill chain stage to login into the target accounts. Therefore the ``credentials`` options needs contain the same hosts that the the ``ACCOUNT_CHANGES`` option

```yaml

    kill_chain:
        PLANNING:
          probability: 1
          starting_network_knowledge:
            credentials:
              ST_PROJ-A-PRV-PC-1:
                username: admin
                password: admin
              ST_PROJ-B-PRV-PC-2:
                username: admin
                password: admin
              ST_PROJ-C-PRV-PC-3:
                username: admin
                password: admin
              ST_INTRA-PRV-RT-DR-1:
                ip_address: 192.168.230.1
                username: admin
                password: admin
              ST_INTRA-PRV-RT-CR:
                ip_address: 192.168.160.1
                username: admin
                password: admin
              REM-PUB-RT-DR:
                ip_address: 192.168.10.2
                username: admin
                password: admin

```

In [ ]:
env.reset()
tap003 = env.game.agents['attacker']
# This while loop will run TAP003 until it hits TAP003 reaches ``ACCESS`` (Therefore passing PLANNING)
while(tap003.current_kill_chain_stage != InsiderKillChain.ACCESS):
    env.step(0)
for account in tap003.network_knowledge['credentials']:
    print(f"----{account}---")
    for key in tap003.network_knowledge['credentials'][account]:
        pprint(f"{key}: {tap003.network_knowledge['credentials'][account][key]}")

#### **Attack Configurations** | kill_chain_settings | Manipulation - Account Changes

|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Action Probability - The chance of successfully carrying out this stage in the kill_chain.|str|_Required_|
|account_changes|List of dictionaries containing the target hostnames, ip and account configuration changes.|list[dict]|_Required_|

```yaml
    kill_chain:
      MANIPULATION:
        probability: 1
        account_changes:
          - host: ST_INTRA-PRV-RT-DR-1
            ip_address: 192.168.230.1 # ST_INTRA-PRV-RT-DR-1
            action: change_password
            username: admin
            new_password: "red_pass"
          - host: ST_INTRA-PRV-RT-CR
            ip_address: 192.168.160.1 # ST_INTRA-PRV-RT-CR
            action: change_password
            username: "admin"
            new_password: "red_pass"
          - host: REM-PUB-RT-DR
            ip_address: 192.168.10.2 # REM-PUB-RT-DR
            action: change_password
            username: "admin"
            new_password: "red_pass"

```


``account_changes``

***

The ``account_changes`` options uses the following schema:

|Option Field |Meaning|Expected Type|Required/Optional|
|-------------|-------|-------------|-----------------|
|host|The hostname of the target host.|str|_Required_|
|ip_address|The IP Address of the target host.|str|_Required_|
|action|The user account manager action. Currently only ``change_password`` is available.|str|_Required_|
|username|The username of the account that TAP003 will target.|str|_Required_|
|new_password|The new password that TAP003 will set.|str|_Required_|

_For further information around the simulation implementation of user sessions please consult the ``user_session_manager`` primAITE API documentation page within PrimAITE's user guide documentation._

#### **Attack Configurations** | kill_chain_settings | Manipulation - Sandbox 

The code cells below can be used as a sandbox to trial out different configuration options and their impact on the simulation. Additionally, if users wish to alter an account other than admin on this sandbox they can alter the ``new_users`` dictionary in the code cell below.

In [ ]:
# New ST_PROJ-B-PRV-PC-2 user account
user_username = "example_user_1"
user_password = "example_pass_1"
user_admin = "False"

In [ ]:
new_users = {"username": user_username, "password": user_password, "is_admin": user_admin }

In [ ]:
# Changing the password
tap003_new_password = "tap003_password"

In [ ]:
account_changes = [{"host":"ST_PROJ-A-PRV-PC-2", "ip_address": "192.168.230.2", "user_name": user_username, "old_password": user_password, "new_password": tap003_new_password}]

Now we update the UC7 config and implement the new user accounts and run through an episode.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
    cfg['simulation']['network']['nodes'][31].update({'users':[]})
    cfg['simulation']['network']['nodes'][31]['users'].append(new_users)
    cfg['agents'][32]['agent_settings']['start_step'] = 1
    cfg['agents'][32]['agent_settings']['frequency'] = 3
    cfg['agents'][32]['agent_settings']['variance'] = 0
    starting_creds = {"username": "admin","password":"admin", "ip_address":"192.168.230.2"} # Adding "ST_PROJ-A-PRV-PC-2" to TAP003's starting_network_knowledge
    cfg['agents'][32]['agent_settings']['kill_chain']['PLANNING']['starting_network_knowledge']['credentials'].update({"ST_PROJ-A-PRV-PC-2": starting_creds})
    cfg['agents'][32]['agent_settings']['kill_chain']['MANIPULATION']['account_changes'] = account_changes
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
while(env.game.step_counter != 90): # 20 Red Actions (frequency of 3)
    env.step(0)
tap003 = env.game.agents['attacker']
target_host = env.game.simulation.network.get_node_by_hostname("ST_PROJ-A-PRV-PC-2")
target_host.user_manager.show()


#### **Attack Configurations** | kill_chain_settings | Exploit - Malicious ACL

|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Action Probability - The chance of successfully carrying out this stage in the kill_chain.|str|_Required_|
|malicious_acls|The configurable ACL that the TAP003 agent adds to the target node.|dict|_Required_|

The malicious ACL is configured identically to the other ACLs. except from the target router/firewall. 
This option is set to the TAP003's configured target host automatically.

TAP003 intends to leverage these ACL's for malicious purposes. The default configuration is to deny all traffic from and towards the 0.0.0.255 subnet. 
The configurability of the malicious ACL will allow for future releases of TAP type agents itself to perform more complex attacks. Currently, the TAP003 aims to block green agent traffic.

``malicious_acl``

***

The ``malicious_acl`` options uses the following schema:

|Option Field |Meaning|Expected Type|Required/Optional|
|-------------|-------|-------------|-----------------|
|target_router|The hostname of target router.|str|_Required_|
|ip_address|The ip_address of target router.|str|_Required_|
|position|The position that the malicious ACL is placed.|str|_Required_|
|permission|The implicit action. Should the ACL ``DENY`` or ``PERMIT``.|str|_Required_|
|src_ip|The source IP address to be targeted by the ``ACL``.|str|_Required_|
|src_wildcard_mask|The wildcard mask for the source ip address.|str|_Required_|
|dst_ip|The destination IP address to be targeted by the ``ACL``.|str|_Required_|
|dst_wildcard_mask|The wildcard mask for the destination ip address.|str|_Required_|
|src_port|The source port to be targeted by the ``ACL``.|str|_Required_|
|dst_port|The destination port to be targeted by the ``ACL``.|str|_Required_|
|protocol_name|The transport layer protocol_name to be targeted.|str|_Required_|

_For further information please consult the simulation ACL documentation within PrimAITE's Sphinx documentation._

The code cell below demonstrates the default ACL's added by TAP003:

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['start_step'] = 1
    cfg['agents'][32]['agent_settings']['frequency'] = 3 # Install action takes multiple timesteps.
    cfg['agents'][32]['agent_settings']['variance'] = 0
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['MANIPULATION']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['ACCESS']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['PLANNING']['probability'] = 1
env = PrimaiteGymEnv(env_config = cfg)
while(env.game.step_counter != 70):
    env.step(0)
tap003 = env.game.agents['attacker']
for key,value in tap003.config.agent_settings:
    if key != 'kill_chain':
        pass
    else:
        for kill_chain_setting_key in value:
                if kill_chain_setting_key == 'EXPLOIT':
                    pprint(value[kill_chain_setting_key]['malicious_acls'][0])
                    pprint(value[kill_chain_setting_key]['malicious_acls'][1])
                    pprint(value[kill_chain_setting_key]['malicious_acls'][2])


In [ ]:
st_intra_prv_rt_dr_1 = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1")
st_intra_prv_rt_dr_1.acl.show()

rem_pub_rt_dr = env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR")
rem_pub_rt_dr.acl.show()

st_intra_prv_rt_cr = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_cr.acl.show()

Unlike the blue agent, TAP003 does not need to use it's action space options for indexing different options, meaning that ACL's are a lot easier to configure.

The sandbox below can be used to try out different configuration options and their impact on the simulation.

#### **Attack Configurations** | kill_chain_settings | Exploit - Sandbox 

In [ ]:
# please ensure that the source/dest ip_addresses are within the blue agent's observation space ip_list option.
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
    cfg['agents'][32]['agent_settings']['start_step'] = 1
    cfg['agents'][32]['agent_settings']['frequency'] = 3
    cfg['agents'][32]['agent_settings']['variance'] = 0
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['position'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['permission'] = "DENY"
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['src_ip'] = "ALL"
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['src_wildcard'] = '0.0.255.255'
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['dst_ip'] = 'ALL'
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['dst_wildcard'] = "0.0.255.255"
    # Please refer to the ``PORT`` class in the primAITE API for more information around the different supported ports
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['src_port'] = 'POSTGRES_SERVER'
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['dst_port'] = "POSTGRES_SERVER"
    cfg['agents'][32]['agent_settings']['kill_chain']['EXPLOIT']['malicious_acls'][0]['protocol_name'] = 'ALL'
env = PrimaiteGymEnv(env_config = cfg)
while(env.game.step_counter != 50):
    env.step(0)
tap003 = env.game.agents['attacker']
target_host = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1")
target_host.acl.show()